# Counts Daily

In [1]:
import sqlite3

In [2]:
import pandas as pd

In [30]:
conn = sqlite3.connect('dpc.db')

df = pd.read_sql('SELECT * FROM count_tweets_daily', conn)

conn.close()

df.head()

,date,count
0,20140101,1
1,20140102,0
2,20140103,0
3,20140104,0
4,20140105,1


In [31]:
def transfer_date(string):
    year = string[:4]
    month = int(string[4:6])
    day = int(string[6:])
    return '/'.join([str(month), str(day), year])

In [32]:
df['date'] = list(map(lambda s: transfer_date(s), df['date']))
df.head()

,date,count
0,1/1/2014,1
1,1/2/2014,0
2,1/3/2014,0
3,1/4/2014,0
4,1/5/2014,1


In [33]:
event_dates = [
    '1/10/2014',
    '2/11/2014',
    '2/27/2014',
    '3/10/2014',
    '3/25/2014',
    '4/22/2014',
    '5/5/2014',
    '6/20/2014',
    '7/14/2014',
    '8/7/2014',
    '9/28/2014',
    '10/8/2014',
    '11/24/2014',
    '12/19/2014',
    '1/22/2016',
    '2/1/2016',
    '3/30/2016',
    '4/13/2016',
    '5/4/2016',
    '6/30/2016',
    '7/12/2016',
    '7/13/2016',
    '8/15/2016',
    '9/15/2016',
    '10/28/2016',
    '11/24/2016',
    '11/25/2016',
    '12/8/2016',
    '12/11/2016',
    '1/5/2018',
    '1/11/2018',
    '2/28/2018',
    '3/8/2018',
    '4/29/2018',
    '5/2/2018',
    '5/15/2018',
    '6/18/2018',
    '6/22/2018',
    '7/6/2018',
    '7/11/2018',
    '7/26/2018',
    '8/2/2018',
    '9/4/2018',
    '10/31/2018',
    '11/10/2018',
    '11/23/2018',
    '12/10/2018',
    '12/19/2018'
]

df['size'] = 0

In [34]:
for i, row in df.iterrows():
    if row['date'] in event_dates:
        df.iloc[[i],[2]] = row['count']
        
df.head(10)

,date,count,size
0,1/1/2014,1,0
1,1/2/2014,0,0
2,1/3/2014,0,0
3,1/4/2014,0,0
4,1/5/2014,1,0
5,1/6/2014,0,0
6,1/7/2014,0,0
7,1/8/2014,1,0
8,1/9/2014,2,0
9,1/10/2014,54,54


In [35]:
df.set_index('date', inplace=True)

In [36]:
df['description'] = ''

In [37]:
df.to_csv('output/PB/daily.csv')

# Anual count

In [53]:
def transfer_date2(string):
    month = int(string[4:6])
    day = int(string[6:])
    return '/'.join([str(month), str(day)])

In [56]:
conn = sqlite3.connect('dpc.db')
df = pd.DataFrame()
df = pd.read_sql('SELECT * FROM count_tweets_daily WHERE date like "2018%"', conn)
df['date'] = list(map(lambda s: transfer_date(s), df['date']))
df.set_index('date', inplace=True)
df.columns = ['Year 2018']

for year in range(2014, 2018):
    y = str(year)
    dft = pd.read_sql('SELECT * FROM count_tweets_daily WHERE date like "'+y+'%" AND date <> "20160229"', conn)
    df['Year '+y] = list(dft['count'])

df.to_csv('output/PB/cmp.csv')
conn.close()

# Top @'s

In [30]:
conn = sqlite3.connect('dpc.db')

df = pd.read_sql('SELECT created_at, text, [extended_tweet.full_text] FROM tweets', conn)

conn.close()

df.head()

,created_at,text,extended_tweet.full_text
0,Wed Jan 01 11:27:40 +0000 2014,Domestic violence up during Christmas.1/4 wome...,None
1,Sun Jan 05 02:09:50 +0000 2014,Dr. Phil and His Viewers Don&amp;#039;t See Ey...,None
2,Wed Jan 08 23:27:19 +0000 2014,Loved all @charliepick's @Mamamia articles so ...,None
3,Thu Jan 09 04:39:49 +0000 2014,@charliepick fantastic article. Well said. But...,None
4,Thu Jan 09 13:35:54 +0000 2014,Well said... I feel a blog post coming on htt...,None


In [37]:
import sys

entities = {
    '2014': {},
    '2015': {},
    '2016': {},
    '2017': {},
    '2018': {}
}

for _, row in df.iterrows():
    year = row['created_at'][-4:]
    if year == '2019':
        break
    if row['extended_tweet.full_text']:
        text = row['extended_tweet.full_text']
    else:
        text = row['text']
    for token in text.split(' '):
        if token.startswith('@'):
            token += '/'
            i = 1
            while token[i].isalpha():
                i += 1
            if i > 1:
                handler = token[:i].lower()
                entities[year][handler] = entities[year].get(handler, 0) + 1

[4, 4, 4, 4, 4]


In [43]:
for year in range(2014, 2019):
    with open('output/handlers/'+str(year)+'.csv', 'w') as f:
        f.write('handler,count\n')
        handlers = entities[str(year)]
        for handler in list(reversed(sorted(handlers, key=handlers.get)))[:100]:
            f.write(handler+','+str(handlers[handler])+'\n')

# Names

In [ ]:
dic = {
    'abcnews':'ABC News',
    'whiteribbonaust':'White Ribbon',
    'qanda': 'ABC Q&A',
    'thetodayshow':'The Today Show',
    'theage':'The Age',
    'acurrentaffair':'A Current Affair (marketplace)',
    'theprojecttv':'The Project (news)',
    'dvvic':'DV Vic',
    'smh':'Sydney Morning Herald',
    'dv':'Demostic Violence',
    'buzzrothfield':'Phil Rothfield (sports journalist)',
    'tonyabbottmhr': 'Tony Abbott',
    'charliepick':'Charlie Pickering (host)',
    'danielandrewsmp': 'Dan Andrews (MP)',
    'rosiebatty': 'Rosie Batty',
    'billshortenmp': 'Bill Shorten',
    'corybernardi':'Cory Bernardi (conservative)',
    'mariska': 'Mariska Hargitay (actress)',
    'mikebairdmp': 'Mike Baird (politician)',
    'colleenhartland': 'Colleen Hartland (MP)',
    'changeaus': 'Change.org',
    'dailylifeau': 'Daily Life',
    'conversationedu': 'The Conversation (news analysis)',
    'familycourtau':'Family Court',
    'womensagenda':"Women's Agenda (news)",
    'turnbullmalcolm': 'Malcolm Turnbull',
    'annastaciamp': 'Annastacia Palaszczuk (MP)',
    'johnmcaldwell': 'John Caldwell (reporter)',
    'lizbroderick':'Elizabeth Broderick (activist)',
    'frichardsonmp': 'Fiona Richardson (MP)',
    'erin': 'Erin Caton (activist)',
    'vanbadham': 'Van Badham (commentator)',
    'kate': 'Kate (artist)',
    'carringtonkl': 'Kerry Carrington (professor)',
    'skinnergj':"Dougy's Daily Digest",
    'newscomauhq':'news.com.au',
    'youtube': 'Youtube',
    'bairdjulia':'Dr Julia Baird (journalist)',
    'paulinehansonoz':'Pauline Hanson (politician)',
    'lyleshelton':'Lyle Shelton (lobbyist)',
    'mirandadevine':'Miranda Devine (columnist)',
    'senatorcash':'Michaelia Cash (minister)',
    'skynewsaust':'Sky News',
    'nrl':'National Rugby League',
    'ourwatchaus':'Our Watch',
    'realmarklatham':'Mark Latham (politician)',
    'nathutchins':'Natalie Hutchins (politician)',
    'peterdutton':'Peter Dutton (MP)',
    'perkinsmiki':'Miki Perkins (editor)',
    'canberratimes':'Canberra Times',
    'dancingceo': 'Dancing CEO (lifestyle)',
    'scottmorrisonmp': 'Scott Morrison'
}

In [ ]:
# df = pd.read_csv('influencers.csv')
df.head()

In [ ]:
# df['name'] = list(map(lambda s: dic[s[1:]], df['handler']))

In [ ]:
df.head()

In [ ]:
# df.to_csv('influencers.csv')